In [28]:
from __future__ import absolute_import
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer 
import argparse
import csv
import io
import logging
import pandas as pd

def get_csv_reader(readable_file = 'Data/reviews - kopie.csv'):
    field_list = ['date', 'comments']
    # Open a channel to read the file from GCS
    #gcs_file = beam.io.filesystems.FileSystems.open(readable_file)
    dataframe = pd.read_csv(readable_file, sep=";", error_bad_lines=False)
    # Return the csv reader
    return dataframe #csv.DictReader(io.TextIOWrapper(gcs_file), field_list)
data = get_csv_reader()

In [29]:
data

,listing_id,id,date,reviewer_id,reviewer_name,comments
0,53671,223358026,1-1-2018,80204789,Adele,Lovely spacious apartment with beautiful priv...
1,55256,223443645,1-1-2018,104838,April,Very large room in a central location. Lotte ...
2,96048,223519984,1-1-2018,157316164,Bobby,Well kept home. Rooms and beds are tiny. Not s...
3,168769,223528554,1-1-2018,14593556,Dave,its a grate place right in center yet quiet an...
4,11600796,223997186,2-1-2018,7904094,Shaun,"Hans gave us a warm welcome, easy check in and..."
...,...,...,...,...,...,...
142,17631393,224047058,2-1-2018,153616329,Roman,This house is amazing everything is clean ther...
143,17674855,224101488,2-1-2018,39886450,Henrik,Roelof was a helpful and responsive host and t...
144,17724135,223993290,2-1-2018,132203039,Luís,The apartment is very noce and cosy. Thank you.
145,17793367,223953109,2-1-2018,156547621,Felix Daniel,"JUST PERFECT! The place, the location, the ho..."


In [30]:
import math
def calculate_sentiment(entry):
    sid_obj = SentimentIntensityAnalyzer() 
    if (type(entry) != str and math.isnan(entry)):
        return -55
    opinion = sid_obj.polarity_scores(entry)
    return opinion['compound']


data['comments'] = data['comments'].apply(calculate_sentiment)
data = data[data['comments'] != -55]
data = data.groupby('date')['comments'].mean()

In [31]:
data

date
1-1-2018    0.771925
2-1-2018    0.639173
Name: comments, dtype: float64

In [ ]:
class MyPredictDoFn(beam.DoFn):

    def __init__(self):
        self._model = SentimentIntensityAnalyzer()
        self.entry = None
        
    def calculate_sentiment(self, entry):
        sid_obj = SentimentIntensityAnalyzer() 
        if (type(self.entry) != str and math.isnan(self.entry)):
            return -55
            opinion = sid_obj.polarity_scores(self.entry)
        return opinion['compound']

    def process(self, elements, **kwargs):

        df = pd.DataFrame(elements)

        df['comments'] = df['comments'].apply(calculate_sentiment)
        df = df[df['comments'] != -55]
        df = df.groupby('date')['comments'].mean()
        
        return df